In [1]:
import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn
import geopandas as gpd
from trino.dbapi import connect
# import folium
import param
import os

import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
from holoviews import opts

from teehr.querying.utils import df_to_gdf

In [2]:
LINE_PLOT_HEIGHT = 400
LINE_PLOT_WIDTH = 800

In [ ]:
# Set global defaults for all line plots
hv.opts.defaults(
    hv.opts.Curve(
        bgcolor="#e7e9ecb8",
        show_grid=True,
        gridstyle={'grid_line_alpha': 0.5, 'grid_line_color': 'white'},
        frame_width=LINE_PLOT_WIDTH,
        frame_height=LINE_PLOT_HEIGHT
    )
)

In [4]:
import warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

In [5]:
# Trino connection configuration
TRINO_HOST = os.environ.get("TRINO_HOST", "localhost")
TRINO_PORT = os.environ.get("TRINO_PORT", 8080)
TRINO_USER = os.environ.get("TRINO_USER", "teehr")
TRINO_CATALOG = os.environ.get("TRINO_CATALOG", "iceberg")
TRINO_SCHEMA = os.environ.get("TRINO_SCHEMA", "teehr")

In [6]:
pn.extension('tabulator', design="material", sizing_mode="stretch_width")

In [ ]:
class DashboardState(param.Parameterized):
    metrics_gdf = param.ClassSelector(class_=gpd.GeoDataFrame, default=None)
    selected_index = param.Integer(default=0)
    location_id = param.String(default="")
    configuration_names = param.List(default=[])
    metrics = param.List(default=[])
    forecast_lead_time = param.List(default=[])
    forecast_lead_time_seconds = param.Integer(default=None)
    forecast_lead_time_options = param.List(default=[])
    timeseries_loading = param.Boolean()
    # selected_configuration_name = param.String(default=None)
    # selected_metric = param.String(default=None)

    @param.depends('selected_index', watch=True)
    def update_location_id(self):
        if hasattr(self, 'metrics_gdf') and self.metrics_gdf is not None:
            try:
                self.location_id = str(self.metrics_gdf.iloc[self.selected_index]['primary_location_id'])
                # print("Setting location_id to:", self.location_id)
            except Exception as e:
                self.location_id = ""
                print("Error occurred while setting location_id.  Setting to empty string.", e)
        else:
            self.location_id = ""
            print("Locations GeoDataFrame not found. Setting location_id to empty string.")

    @param.depends('metrics_gdf', watch=True)
    def update_metrics(self):
        if hasattr(self, 'metrics_gdf') and self.metrics_gdf is not None:
            self.configuration_names = self.metrics_gdf['configuration_name'].unique().tolist()
            self.metrics = self.metrics_gdf.columns.difference(['geometry', 'primary_location_id', 'name', 'configuration_name', 'forecast_lead_time', 'forecast_lead_time_display', 'forecast_lead_time_seconds']).tolist()

            # Initialize with first configuration
            self._update_forecast_lead_times(self.configuration_names[0] if self.configuration_names else None)
        else:
            self.configuration_names = []
            self.metrics = []
            self.forecast_lead_time = []
            self.forecast_lead_time_options = []

    def _update_forecast_lead_times(self, configuration_name):
        """Update forecast lead time options for a specific configuration."""
        if not configuration_name or self.metrics_gdf is None:
            self.forecast_lead_time = []
            self.forecast_lead_time_options = []
            return

        # Get lead times for the specific configuration
        lead_times_data = self.metrics_gdf[
            self.metrics_gdf['configuration_name'] == configuration_name
        ][['forecast_lead_time_seconds', 'forecast_lead_time_display']].drop_duplicates()

        self.forecast_lead_time = sorted(lead_times_data['forecast_lead_time_seconds'].unique().tolist())

        # Store display mapping for widget
        self.forecast_lead_time_display_map = dict(
            zip(lead_times_data['forecast_lead_time_seconds'],
                lead_times_data['forecast_lead_time_display'])
        )

        # Create simple display options (just strings, not tuples)
        self.forecast_lead_time_options = [
            self.forecast_lead_time_display_map.get(val, f"{val} seconds")
            for val in self.forecast_lead_time
        ]

        # Also create a reverse mapping from display to seconds
        self.forecast_lead_time_reverse_map = dict(
            zip([self.forecast_lead_time_display_map.get(val, f"{val} seconds") for val in self.forecast_lead_time],
                self.forecast_lead_time)
        )

        # Set initial forecast_lead_time_seconds
        if self.forecast_lead_time:
            self.forecast_lead_time_seconds = self.forecast_lead_time[0]


state = DashboardState()

In [8]:

def get_trino_connection():
    """Establishes and returns a Trino database connection."""
    conn = connect(
        host=TRINO_HOST,
        user=TRINO_USER,
        catalog=TRINO_CATALOG,
        schema=TRINO_SCHEMA,
        http_scheme='http',
        port=TRINO_PORT,
        # For production, add authentication:
        # auth=BasicAuthentication("username", "password")
    )
    return conn


In [9]:
# Selected location callback
def selection_callback(index):
    if len(index) > 1:
        print("Multiple selection detected, using the first selected index.")
    state.timeseries_loading = True
    state.selected_index = index[0]
    # print("Selected index:", state.selected_index)
    # You can also get the selected rows:
    # selected_points = gdf_prj.iloc[state.selected_index]
    # Do something with selected_points

In [ ]:
# def get_locations():
#     """Fetches location data from the Trino database and returns it as a GeoDataFrame."""
#     conn = get_trino_connection()
#     query = """
#     SELECT * FROM iceberg.teehr.locations
#     WHERE id LIKE 'usgs-%'
#     """
#     df = pd.read_sql(query, conn)
#     gdf = df_to_gdf(df)
#     return gdf

# locations_gdf = get_locations()
# state.locations_gdf = locations_gdf

In [ ]:
def get_metrics_by_location(forecast_lead_time: str = None):
    """Fetches simulation metrics by location from the Trino database and returns it as a GeoDataFrame."""
    conn = get_trino_connection()
    # Build the SQL query with optional forecast_lead_time filter
    if forecast_lead_time:
        sql = f"""
        SELECT *,
               CASE
                   WHEN forecast_lead_time >= 86400 THEN
                       FORMAT('%d days %d hours',
                              forecast_lead_time / 86400,
                              (forecast_lead_time % 86400) / 3600)
                   WHEN forecast_lead_time >= 3600 THEN
                       FORMAT('%d hours', forecast_lead_time / 3600)
                   ELSE
                       FORMAT('%d minutes', forecast_lead_time / 60)
               END as forecast_lead_time_display,
               forecast_lead_time as forecast_lead_time_seconds
        FROM iceberg.teehr.forecast_metrics_by_location
        WHERE forecast_lead_time = {forecast_lead_time}
        """
    else:
        sql = """
        SELECT *,
               CASE
                   WHEN forecast_lead_time >= 86400 THEN
                       FORMAT('%d days %d hours',
                              forecast_lead_time / 86400,
                              (forecast_lead_time % 86400) / 3600)
                   WHEN forecast_lead_time >= 3600 THEN
                       FORMAT('%d hours', forecast_lead_time / 3600)
                   ELSE
                       FORMAT('%d minutes', forecast_lead_time / 60)
               END as forecast_lead_time_display,
               forecast_lead_time as forecast_lead_time_seconds
        FROM iceberg.teehr.forecast_metrics_by_location
        """
    df = pd.read_sql(sql, conn)
    gdf = df_to_gdf(df)

    return gdf

metrics_gdf = get_metrics_by_location()
state.metrics_gdf = metrics_gdf

In [ ]:
def get_primary_timeseries(location_id: str):
    """Fetches primary timeseries data for a given location from the Trino database and returns it as a DataFrame."""
    conn = get_trino_connection()
    sql = f"SELECT * FROM iceberg.teehr.primary_timeseries WHERE location_id = '{location_id}'"
    df = pd.read_sql(sql, conn)

    return df
# get_primary_timeseries("usgs-01347000")

In [ ]:
def get_secondary_timeseries(location_id: str, configuration_names: list = ["nwm30_short_range"]):
    """Fetches secondary timeseries data for a given primary location from the Trino database and returns it as a DataFrame."""
    conn = get_trino_connection()
    sql = f"""
        SELECT st.*
        FROM iceberg.teehr.secondary_timeseries st
        JOIN location_crosswalks lc
        ON st.location_id = lc.secondary_location_id
        WHERE lc.primary_location_id = '{location_id}'
        AND configuration_name IN ({','.join(f"'{name}'" for name in configuration_names)})
    """
    df = pd.read_sql(sql, conn)

    return df
# get_secondary_timeseries("usgs-01347000")

In [ ]:
def get_timeseries_plot(location_id: str, configuration_name: str = None):
    if not location_id:
        return None
    # Use the passed configuration_name or fallback to first available
    if not configuration_name and state.configuration_names:
        configuration_name = state.configuration_names[0]

    primary_df = get_primary_timeseries(location_id)
    secondary_df = get_secondary_timeseries(
        location_id, configuration_names=[configuration_name]
    )

    min_time = secondary_df['value_time'].min()
    max_time = secondary_df['value_time'].max()

    primary_plot = primary_df[
        primary_df['value_time'].between(min_time, max_time)
    ].hvplot.line(
        x='value_time',
        y='value',
        by='configuration_name',
        legend=False,
        color='black'
    )

    secondary_plot = secondary_df[
        secondary_df['value_time'].between(min_time, max_time)
    ].hvplot.line(
        x='value_time',
        y='value',
        by=['configuration_name', 'reference_time'],
        legend=False
    )

    state.timeseries_loading = False

    return (primary_plot * secondary_plot).opts(
        title=f"Observed and Simulated Timeseries at {location_id}"
    )
# get_timeseries_plot("usgs-01347000")

In [ ]:
def get_locations_map(
    metrics_gdf: gpd.GeoDataFrame = None,
    configuration_name: str = None,
    metric: str = None,
    forecast_lead_time_seconds: int = None
):
    # First filter by forecast lead time if specified
    if forecast_lead_time_seconds and metrics_gdf is not None:
        gdf = metrics_gdf[metrics_gdf['forecast_lead_time_seconds'] == forecast_lead_time_seconds]
    else:
        gdf = metrics_gdf

    # Then filter by configuration
    if configuration_name and gdf is not None:
        gdf = gdf[gdf['configuration_name'] == configuration_name]
    elif gdf is not None and len(state.configuration_names) > 0:
        gdf = gdf[gdf['configuration_name'] == state.configuration_names[0]]

    if not metric:
        metric = state.metrics[0]

    # Normalize metric between 0 and 1 for color mapping
    gdf_plot = gdf.copy()
    # metric_values = gdf_plot[metric].values
    # metric_min = metric_values.min()
    # metric_max = metric_values.max()
    # # Avoid division by zero
    # if metric_max != metric_min:
    #     normalized_metric = f"{metric}_normalized"
    #     gdf_plot[normalized_metric] = (metric_values - metric_min) / (metric_max - metric_min)
    # else:
    #     # All values are the same
    #     normalized_metric = f"{metric}_normalized"
    #     gdf_plot[normalized_metric] = 0.5  # Set to middle value

    # print(f"Metric values range: {metric_min} to {metric_max}")
    # print(f"Normalized values range: {gdf_plot[normalized_metric].min()} to {gdf_plot[normalized_metric].max()}")
    # print(f"NaN count: {gdf_plot[normalized_metric].isna().sum()}")

    tiles = gv.tile_sources.OSM
    gdf_prj = gdf_plot.to_crs("EPSG:3857")

    points = gv.Points(
        gdf_prj,
        crs=ccrs.GOOGLE_MERCATOR,
        vdims=[metric, "primary_location_id"],
    ).opts(
        tools=["tap", "hover"],
        color=metric,
        size=10,
        width=800,
        height=500,
        selection_alpha=1,
        nonselection_alpha=0.4,
        selection_color="red",
        # cnorm="log",
        cmap="plasma",
    )
    # Attach a selection stream
    selection = hv.streams.Selection1D(source=points)
    # Register the callback
    selection.add_subscriber(selection_callback)

    return (tiles * points) #.opts(width=600, height=500)

In [ ]:
def selected_location_metrics_table(location_id, forecast_lead_time_seconds=None):
    # Show metrics for selected location by configuration_name, transposed for easier viewing
    if not location_id:
        return pn.pane.Markdown("Select a location to view its metrics.")

    filtered_gdf = state.metrics_gdf[state.metrics_gdf['primary_location_id'] == location_id]

    # Filter by forecast lead time if specified
    if forecast_lead_time_seconds:
        filtered_gdf = filtered_gdf[filtered_gdf['forecast_lead_time_seconds'] == forecast_lead_time_seconds]
        title_suffix = f" (Forecast Lead Time: {state.forecast_lead_time_display_map.get(forecast_lead_time_seconds, f'{forecast_lead_time_seconds} seconds')})"
    else:
        title_suffix = " (All Forecast Lead Times)"

    if filtered_gdf.empty:
        return pn.pane.Markdown(f"No metrics found for location ID: {location_id}{title_suffix}")

    # Drop geometry and lead time columns for cleaner display
    display_gdf = filtered_gdf.drop(columns=['geometry'])
    display_gdf = display_gdf.drop(
        columns=[
            'forecast_lead_time_seconds',
            'forecast_lead_time_display',
            'forecast_lead_time',
            'primary_location_id'
        ],
        errors='ignore'
    )

    name = display_gdf.iloc[0]['name'] if 'name' in display_gdf.columns else location_id
    display_gdf = display_gdf.drop(columns=['name'], errors='ignore')

    # Transpose for easier viewing
    transposed_gdf = display_gdf.set_index('configuration_name').T.reset_index()

    table = pn.Column(
        pn.pane.Markdown(f"## Metrics for {location_id}: {title_suffix}\n ### {name}"),
        pn.widgets.Tabulator(
            transposed_gdf,
            theme="bootstrap",
            pagination='remote',
            page_size=10,
            sizing_mode='stretch_width',
        )
    )

    return table


In [ ]:
def get_location_details_title(location_id: str, loading: bool = False):
    if not location_id:
        return pn.pane.Markdown("# Location Details Pane")
    if not loading:
        return pn.pane.Markdown(f"# Location Details for {location_id}")
    return pn.Column(
        pn.pane.Markdown(f"# Location Details for {location_id}"),
        pn.indicators.LoadingSpinner(value=True, size=40, name='Loading...')
    )

In [ ]:
locations_widget = pn.widgets.Select(
    name="location",
    options=list(state.metrics_gdf['primary_location_id']),
    value=state.location_id,
)
metrics_widget = pn.widgets.Select(
    name="metric",
    options=list(state.metrics),
    value=state.metrics[0],
)
configurations_widget = pn.widgets.Select(
    name="configuration",
    options=list(state.configuration_names),
    value=state.configuration_names[0],
)
forecast_lead_time_widget = pn.widgets.Select(
    name="forecast_lead_time",
    options=state.forecast_lead_time_options,
    value=state.forecast_lead_time_options[0] if state.forecast_lead_time_options else None,
)

bound_timeseries_plot = pn.bind(
    get_timeseries_plot,
    location_id=state.param.location_id,
    configuration_name=configurations_widget.param.value
)
bound_locations_map = pn.bind(
    get_locations_map,
    metrics_gdf=state.param.metrics_gdf,
    configuration_name=configurations_widget.param.value,
    metric=metrics_widget.param.value,
    forecast_lead_time_seconds=state.param.forecast_lead_time_seconds
)
bound_selected_location_metrics_table = pn.bind(
    selected_location_metrics_table,
    location_id=state.param.location_id,
    forecast_lead_time_seconds=state.param.forecast_lead_time_seconds
)
bound_location_details_title = pn.bind(
    get_location_details_title,
    location_id=state.param.location_id,
    loading=state.param.timeseries_loading
)
@pn.depends(forecast_lead_time_widget.param.value, watch=True)
def sync_forecast_lead_time(selected_lead_time_display):
    """Sync widget selection with state parameter."""
    if selected_lead_time_display and hasattr(state, 'forecast_lead_time_reverse_map'):
        # Convert display string back to seconds
        forecast_lead_time_value = state.forecast_lead_time_reverse_map.get(selected_lead_time_display)
        if forecast_lead_time_value:
            state.forecast_lead_time_seconds = forecast_lead_time_value
sync_forecast_lead_time(forecast_lead_time_widget.value)

# Bind the widget options to update automatically
@param.depends(state.param.forecast_lead_time_options, watch=True)
def update_forecast_lead_time_widget_options(forecast_lead_time_options):  # Add parameter here
    """Update forecast lead time widget options when they change."""
    forecast_lead_time_widget.options = state.forecast_lead_time_options
    # Reset to first option if current selection is not available
    if (forecast_lead_time_widget.value not in state.forecast_lead_time_options
        and state.forecast_lead_time_options):
        forecast_lead_time_widget.value = state.forecast_lead_time_options[0]
update_forecast_lead_time_widget_options(state.forecast_lead_time_options)

# Update location widget when metrics change
@param.depends(state.param.metrics_gdf, watch=True)
def update_location_widget_options(metrics_gdf):  # Add parameter here
    """Update location widget options when metrics_gdf changes."""
    if state.metrics_gdf is not None:
        new_options = list(state.metrics_gdf['primary_location_id'].unique())
        locations_widget.options = new_options

        # Reset location selection if current selection is not in new data
        if locations_widget.value not in new_options and len(new_options) > 0:
            locations_widget.value = new_options[0]
update_location_widget_options(state.metrics_gdf)

@pn.depends(configurations_widget.param.value, watch=True)
def update_forecast_lead_times_for_config(selected_config):
    """Update forecast lead time options when configuration changes."""
    if selected_config and hasattr(state, '_update_forecast_lead_times'):
        state._update_forecast_lead_times(selected_config)

update_forecast_lead_times_for_config(configurations_widget.value)

tabulator = pn.widgets.Tabulator(
    state.metrics_gdf.drop(columns=["geometry"]),
    theme="bootstrap",
    pagination="remote",
    page_size=10
)
@pn.depends(state.param.metrics_gdf, watch=True)
def update_tabulator(metrics_gdf):  # Add the parameter here
    if metrics_gdf is not None:
        tabulator.value = metrics_gdf.drop(
            columns=["geometry", "forecast_lead_time", "forecast_lead_time_seconds"]
        )
update_tabulator(state.metrics_gdf)

top_pane = pn.Tabs(
    (
        "Location Metrics Map",
        pn.Column(
            pn.Row(
                configurations_widget,
                metrics_widget,
                forecast_lead_time_widget
            ),
            pn.Row(bound_locations_map)
        )
    ),
    ("Location Metrics List", tabulator)
)

sidebar = pn.Column(
    pn.pane.Markdown(f"""
        ### 🔧 Database Connection Information
        ```
        Host: {TRINO_HOST}
        Port: {TRINO_PORT}
        User: {TRINO_USER}
        Catalog: {TRINO_CATALOG}
        Schema: {TRINO_SCHEMA}
        ```
    """),
    "---",
)

main = pn.Column(
    top_pane,
    bound_location_details_title,
    bound_timeseries_plot,
    bound_selected_location_metrics_table
)

pn.template.MaterialTemplate(
    site="TEEHR",
    title="Forecast Dashboard",
    sidebar=[sidebar],
    main=[main],
).servable()

MaterialTemplate
    [js_area] HTML(None, design=<class 'panel.theme.materi..., height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] MaterialTemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-13027166256] Tabs(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi..., name='Location Metrics Map', sizing_mode='stretch_width')
        [1] Tabulator(design=<class 'panel.theme.materi..., page_size=10, pagination='remote', sizing_mode='stretch_width', theme='bootstrap', value=  primary_location_id   co...)
    [main-13036166416] ParamFunction(function, _pane=Markdown, defer_load=False, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [nav-12919844080] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [1] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [2] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [3] Select(design=<class 'panel.theme.materi..., name='location', options=['usgs-03068800', ...], sizing_mode='stretch_width', value='usgs-03068800')
        [4] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [5] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [6] Select(design=<class 'panel.theme.materi..., name='configuration', options=['nwm30_retrospective'], sizing_mode='stretch_width', value='nwm30_retrospective')
        [7] Select(design=<class 'panel.theme.materi..., name='metric', options=['average', 'count', ...], sizing_mode='stretch_width', value='average')